# RAG desde PDF - ESPAÑOL

Este RAG recibe tres pdfs sobre animales y responde a unas preguntas

## 1. Código

In [1]:
# Introducción documento "externo"
import PyPDF2
pdf_names=[ "arboles", "flores", "plantas"]

pdf_paths=[]
for name in pdf_names:
    pdf_paths.append('pdfs/'+name+'.pdf')
text=""
for pdf in pdf_paths:
    with open(pdf, 'rb') as pdf_file:
        reader = PyPDF2.PdfReader(pdf_file)
        for page in reader.pages:
            text = text + page.extract_text() 

In [2]:
# Split del texto recibido
import langchain
from langchain.text_splitter  import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n",  
    chunk_size=400,  
    # chunk_overlap=200 
)

splits = text_splitter.split_text(text)
print(len(splits))

142


In [4]:
#Vectorizar
from langchain_mongodb import MongoDBAtlasVectorSearch
from pymongo import MongoClient
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.schema import Document
from dotenv import load_dotenv
import os 

load_dotenv()

MONGODB_ATLAS_CLUSTER_URI = api_key = os.getenv("MONGODB_ATLAS_CLUSTER_URI")
DB_NAME = os.getenv("DB_NAME")
COLLECTION_NAME = os.getenv("COLLECTION_NAME")
ATLAS_VECTOR_SEARCH_INDEX_NAME = os.getenv("ATLAS_VECTOR_SEARCH_INDEX_NAME")

client = MongoClient(MONGODB_ATLAS_CLUSTER_URI)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
MONGODB_COLLECTION = client[DB_NAME][COLLECTION_NAME]

vector_store = MongoDBAtlasVectorSearch(
    collection=MONGODB_COLLECTION,
    embedding=embeddings,
    index_name=ATLAS_VECTOR_SEARCH_INDEX_NAME,
    relevance_score_fn="cosine",
)

for split in splits:
    embedding = embeddings.embed_documents(split) 
    document = Document(
        page_content=split,  
        embedding=embedding 
    )

    vector_store.add_documents([document])
vector_store.create_vector_search_index(dimensions=768)


retriever = vector_store.as_retriever()

ServerSelectionTimeoutError: SSL handshake failed: cluster0-shard-00-02.wbknj.mongodb.net:27017: [SSL: TLSV1_ALERT_INTERNAL_ERROR] tlsv1 alert internal error (_ssl.c:1018) (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms),SSL handshake failed: cluster0-shard-00-00.wbknj.mongodb.net:27017: [SSL: TLSV1_ALERT_INTERNAL_ERROR] tlsv1 alert internal error (_ssl.c:1018) (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms),SSL handshake failed: cluster0-shard-00-01.wbknj.mongodb.net:27017: [SSL: TLSV1_ALERT_INTERNAL_ERROR] tlsv1 alert internal error (_ssl.c:1018) (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 67616fbac7c60c7e08df64f4, topology_type: ReplicaSetNoPrimary, servers: [<ServerDescription ('cluster0-shard-00-00.wbknj.mongodb.net', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('SSL handshake failed: cluster0-shard-00-00.wbknj

In [ ]:
# Ollama 
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

# Prompt
template = """Responde a la pregunta concisamente y en castellano en base a este contexto:
{context}

Pregunta: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# Local LLM
ollama_llm = "llama3.2"
model_local = ChatOllama(model=ollama_llm)

# Chain
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model_local
    | StrOutputParser()
)


: 

## 2. Preguntas

In [ ]:
chain.invoke("¿Dime cómo se puede reproducir una planta?")

: 

In [ ]:
chain.invoke("¿Cómo es la estructura de un árbol?")

: 

In [ ]:
chain.invoke("¿Cuál es la función del tronco y las ramas?")

: 

In [ ]:
chain.invoke("¿Tienen las cabras pupilas circulares?")

: 

In [ ]:
chain.invoke("Háblame de la clasificación de las flores")

: 

In [ ]:
chain.invoke("¿Un árbol emite nitrógeno a la atmósfera?")

: 